In [1]:
import copy
import json
import math
import numpy as np
import pprint as pp
import random
import indyconfig
import indyjson

### Connect to Indy by IndyDCP

In [2]:
import indydcp_client as client

In [3]:
bind_ip   = "192.168.3.130"
server_ip = "192.168.3.129"
name = "NRMK-Indy7"
indy = client.IndyDCPClient(bind_ip, server_ip, name)
indy.connect()

Connect: Bind IP (192.168.3.130), Server IP (192.168.3.129)


True

In [4]:
indy.go_home()

### Set Configuration

In [12]:
box_size = [3,2]
obj_num_per_box = 3
max_obj_num = box_size[0] * box_size[1] * obj_num_per_box

obj_space_x = [0.404, 0.654]
obj_space_y = [-0.186, 0.1-0.086]
tar_space_x = [0.204, 0.404]
tar_space_y = [-0.386, -0.186]
#tar_space_x = [212, 312]
#tar_space_y = [365, 455]

In [13]:
# Set Workspace Configuration
# 따로 object space, target space 지정해 주지 않으면, 같은 공간으로 설정됨
# box_size따로 기입안해주면 에러
workspace = indyconfig.WorkSpace(obj_space_x,
                                 obj_space_y,
                                 tar_space_x, 
                                 tar_space_y,
                                box_size)

# Set Object Configuration
# Workspace바뀌면 다시 해야함!
# STACK_TASK = 0
# PALLETIZE_TASK = 1
obj_config = indyconfig.TaskObjConfig(workspace,indyconfig.PALLETIZE_TASK, class_num = 1)
obj_config.create_random_obj(max_obj_num=max_obj_num,
                             min_obj_num=3)
print("물체 개수: %d" %obj_config.obj_num)

# command_config 객체 생성
command_config = indyjson.CommandConfig()

# json 파일 생성하는 객체 생성
JsonPro = indyjson.IndyMakeJsonProgram()

물체 개수: 17


In [14]:
pp.pprint(obj_config.obj)
pp.pprint(obj_config.tar_pos)

[{'class_': 1, 'pos_': [0.5249206362437648, -0.07610747802979892, 0.46]},
 {'class_': 1, 'pos_': [0.48330966127429664, -0.16011455125788202, 0.46]},
 {'class_': 1, 'pos_': [0.5835638546528508, -0.06062259453404201, 0.46]},
 {'class_': 1, 'pos_': [0.412950640353274, -0.12949324903229453, 0.46]},
 {'class_': 1, 'pos_': [0.5339998504947157, -0.14068821025566092, 0.46]},
 {'class_': 1, 'pos_': [0.6405587856178958, -0.17462170524393722, 0.46]},
 {'class_': 1, 'pos_': [0.49091375366074314, -0.03284140653670145, 0.46]},
 {'class_': 1, 'pos_': [0.4493145458411132, -0.05204467824994197, 0.46]},
 {'class_': 1, 'pos_': [0.5020580984357823, -0.11929325771523513, 0.46]},
 {'class_': 1, 'pos_': [0.5467004014133505, -0.04228232703617946, 0.46]},
 {'class_': 1, 'pos_': [0.5100464508535087, 0.006302904375551727, 0.46]},
 {'class_': 1, 'pos_': [0.6151168592277703, -0.09375258759147749, 0.46]},
 {'class_': 1, 'pos_': [0.4197356923294072, -0.14590315010718058, 0.46]},
 {'class_': 1, 'pos_': [0.60764175028

### Palletizing Task Logic

In [15]:
def move_command(obj_config, command_config, JsonPro, pos_xyz) :
    command_config.create_radnom_command_config()
    pos_xyz = copy.deepcopy(pos_xyz)
    pos_xyz.extend([0,180,0])
    command_config.p = pos_xyz
    ##joint 모드일 경우 q도 추가하는 구문 만들어야함 : 이때는 p->q로 만드는 함수 필요
    JsonPro.add_seq(command_config)


# pick & place json파일로 만드는 함수 구현 해야함!(IndyJsonMake함수에서 ㄱㄱㄱ)
def pick_command():
    pass

def place_command():
    pass

for i, obj_ in enumerate(obj_config.obj):
    for j in range(4):
        if j == 0 :# obj로 가기
            move_command(obj_config,command_config, JsonPro, obj_['pos_'])

        elif j == 1: #pick하기(놓고 다시 위치로 돌아오는 것 까지)
            #obj까지 내려가기
            pos_down = copy.deepcopy(obj_['pos_'])
            pos_down[-1] = indyconfig.OBJ_POS_Z+indyconfig.OBJ_SIZE_Z
            move_command(obj_config,command_config, JsonPro, pos_down )
            
            #pick
            pick_command()
            
            #다시 올라오기
            move_command(obj_config,command_config, JsonPro, obj_['pos_'] )
            
        elif j == 2: #target위치로 가기
            # 모든 박스에 하나씩 담고, 또 담기
            move_command(obj_config, command_config, JsonPro, obj_config.tar_pos[i%(box_size[0]*box_size[1])] )
            
            # 각 박스를 모두 채우고 다음 박스 담기
           # move_command(obj_config, command_config, JsonPro, obj_config.tar_pos[math.floor(i/obj_num_per_box)] )

        else : #place하기
            #제일 높은 obj까지 내려가기
            
            # 모든 박스에 하나씩 담고, 또 담기
            pos_down = copy.deepcopy(obj_config.tar_pos[i%(box_size[0]*box_size[1])])
            pos_down[-1] = indyconfig.OBJ_POS_Z + (indyconfig.OBJ_SIZE_Z * (math.ceil((i/(box_size[0]*box_size[1])))+1))
            
            # 각 박스를 모두 채우고 다음 박스 담기
            # pos_down = copy.deepcopy(obj_config.tar_pos[math.floor(i/obj_num_per_box)])
            # pos_down[-1] = indyconfig.OBJ_POS_Z + (indyconfig.OBJ_SIZE_Z * (i%obj_num_per_box))
            
            
            move_command(obj_config, command_config, JsonPro, pos_down )
            
            #pick
            place_command()
            
            #다시 올라오기
            move_command(obj_config, command_config, JsonPro, obj_config.tar_pos[obj_['class_']-1] )
    

In [16]:
JsonPro.JsonPro

{'program': [{'type': 999,
   'enable': True,
   'palletInfo': [],
   'toolInfo': [],
   'collisionPolicy': {'time': 2, 'policy': 0},
   'pId': 0,
   'id': 1},
  {'type': 2, 'varList': [], 'enable': True, 'pId': 0, 'id': 2},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_VELO5.0_01',
   'id': 3},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_VELO9.0_01',
   'id': 4},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_VELO5.0_02',
   'id': 5},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_VELO9.0_02',
   'id': 6},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_VELO5.0_03',
   'id': 7},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_VELO9.0_03',
   'id': 8},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_VELO5.0_04',
   'id': 9},
  {'type': 103,
   'enable': True,
   'pId': 0,
   'name': 'Frame_move_to_V

In [17]:
indy.set_and_start_json_program(json.dumps(JsonPro.JsonPro))

In [ ]:
with open('palletize_test.7.json', 'w') as fp:
    json.dump(JsonPro.JsonPro, fp)

In [4]:
indy.go_home()

timeout: timed out